<a href="https://colab.research.google.com/github/rondi91/BISINDO/blob/main/8_BISINDO_HARI_97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.PREPARING THE WORKSPACE**

## 1.1 Mount Google Drive.

In [ ]:
# 1.1 Mount Google Drive.

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Tensorflow version
import tensorflow as tf 
print(tf.__version__)

## 1.2 Prepare & Clone Template TFOD

In [ ]:

# buat folder project baru 
NEW_folder_work = '23.BISINDO_NAMA_HARI'
# buat base_url root
BASE_PATH = '/content/gdrive/MyDrive/'+NEW_folder_work+'/Tensorflow'

# Clone template Train TFOD
%cd /content/gdrive/MyDrive/
!mkdir {NEW_folder_work}
%cd /content/gdrive/MyDrive/{NEW_folder_work}
!git clone https://github.com/rondi91/Tensorflow.git
 



##  **1.3 Set Up Variable Path**

In [ ]:
# Set Up Variable Path

NUM_STEPS = 2000
LR_TOTAL_STEPS =100000
BATCH_SIZE=64
CUSTOM_MODEL_NAME = 'my_ssd_mobnet_'           #folder Train project
OUTPUT_MODEL= 'my_ssd_mobnet_'+ str(NUM_STEPS)                     #OUTPUT MODEL TRAIN

CUSTOM_MODEL_NAME+ str(NUM_STEPS)
WORKSPACE_PATH = BASE_PATH+ '/workspace'
SCRIPTS_PATH =  WORKSPACE_PATH+'/scripts'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
EXPORTED_MODELS = WORKSPACE_PATH+'/exported-models'

# my_ssd_mobnet5_100batch_5k

#APIMODEL_PATH = WORKSPACE_PATH+ 'Tensorflow/models'
#CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
#CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_resnet101_v1_fpn/'

# **2.Instalation TFOD**

## **2.1 Install Object Detection**

In [ ]:
# Clone the tensorflow models git repository

%cd /content
!git clone https://github.com/tensorflow/models.git

In [ ]:
# Install TensorFlow Object Detection API

%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=in-tree-build .

In [ ]:
!cp {ANNOTATION_PATH+'/arial.ttf'} /usr/share/fonts/truetype
!cp {ANNOTATION_PATH+'/arial.ttf'} /usr/local/lib/python3.7/dist-packages/object_detection/utils

## **2.2 Test the model builder**

In [ ]:
#Test Instalation objec detection
!python object_detection/builders/model_builder_tf2_test.py

### **finish instalation**

#  **3.Preprocessing Dataset**

##  3.1 Clone Dataset Image 

In [ ]:
#Clone dataset image 

#cd into tensorflow/workspace
%cd {BASE_PATH}/workspace/

!git clone https://github.com/rondi91/images.git



## **3.2 Create the CSV files and the "label_map.pbtxt" file**

Current working directory is /mydrive/customTF2/data/

Run xml_to_csv script below to create ***test_labels.csv*** and ***train_labels.csv***

This also creates the ***label_map.pbtxt*** file using the classes mentioned in the xml files. 

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

%cd {IMAGE_PATH}

#adjusted from: https://github.com/datitran/raccoon_dataset
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,   
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['ALL_DATASET']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{ANNOTATION_PATH}/{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join(ANNOTATION_PATH+"/label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')    

### optional CEK DATASET

In [ ]:
import pandas as pd
data = pd.read_csv(ANNOTATION_PATH+"/ALL_DATASET.csv")
data.shape

In [ ]:
data.head(20)

In [ ]:
jml_class=data['class'].drop_duplicates().count()
# class_name=data['class'].drop_duplicates()
class_name=data.groupby('class')['class'].count()

print('======class name ============')
print(class_name)
print('======class total =====')
print(jml_class)

## **3.3 SPLIT & TEST**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

%cd {ANNOTATION_PATH}
data_all= pd.read_csv(ANNOTATION_PATH+'/ALL_DATASET.csv')

X=data_all.drop('class', axis=1)
y=data_all['class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1010)

y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)

data_train=pd.merge(X_train,y_train,left_index=True,right_index=True)
data_test=pd.merge(X_test,y_test,left_index=True,right_index=True)

data_train.to_csv('train.csv')
print('Successfully created train.csv ')
data_test.to_csv('test.csv')
print('Successfully created test.csv ') 




In [ ]:
print('dimensi train.csv ')
train = pd.read_csv(ANNOTATION_PATH+"/train.csv")
train.shape 

In [ ]:
test = pd.read_csv(ANNOTATION_PATH+"/test.csv")
test.shape 

## 3.4 GENERATE TF RECORD

In [ ]:
#Usage:  
#!python generate_tfrecord.py output.csv output_pb.txt /path/to/images output.tfrecords
%cd {ANNOTATION_PATH}
#For train.record
!python {SCRIPTS_PATH}/generate_tfrecord.py  train.csv  label_map.pbtxt {IMAGE_PATH}/ALL_DATASET/ train.record

#For test.record
!python {SCRIPTS_PATH}/generate_tfrecord.py  test.csv  label_map.pbtxt {IMAGE_PATH}/ALL_DATASET/ test.record

NOTE:

If you haven't downloaded any pre-trained model yet, go back to Step 1 and finish downloading any pre-trained model of your choice.

We are almost ready to start our model training, just a few more steps before we start our model training!

# **4.BUILD MODEL PROCESS**

## 4.1 Download TF Models Pretrained Models from Tensorflow Model Zoo

In [ ]:
# Step 6 Download TF Models Pretrained Models from Tensorflow Model Zoo


# download Pretrained Models
!wget -O {PRETRAINED_MODEL_PATH}/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
# ekstrak file to Pretraine models dir
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz


In [ ]:
# 3.5.1 Copy Model Config to Training Folder
!mkdir {MODEL_PATH +'/'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}


## 4.2 Setting  Config Pipeline 

In [ ]:
# 3.5.2 Update Config For Transfer Learning
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
#3.5.3 show config pipeline
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
#3.5.4 Set config pipeline
pipeline_config.model.ssd.num_classes = jml_class
pipeline_config.train_config.batch_size = BATCH_SIZE
pipeline_config.train_config.num_steps = NUM_STEPS
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = LR_TOTAL_STEPS
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [ ]:
#3.5.5 show config pipeline setelah disetting
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

Step 12- Copying some files


*   Copy the "model_main_tf2.py" file from   "TensorFlow\models\research\object_detection" and paste it into training_demo. We will need this file for training the model.

*   Copy the "exporter_main_v2.py" file from "TensorFlow\models\research\object_detection" and paste it into training_demo.
We will need this file to export the trained model






Step 13- Configure the pipeline file.

Refer the mentioned [Medium article](https://medium.com/@nisargkapkar/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc?source=friends_link&sk=0bb205df0e1c29a2e78c28671ddf4494) for more details!

In [ ]:
#Reusing TensorBoard on port 6006 (pid 554)
!kill 554

# **5.MODEL EVALUATION**

In [ ]:
#start the Tensorboard
%load_ext tensorboard

%tensorboard --logdir={MODEL_PATH +'/'+CUSTOM_MODEL_NAME}

# %load_ext tensorboard
# %tensorboard --logdir=models/[name_of_pre-trained-model_you_downloaded]

In [ ]:
# Reinstal OpenCV
!pip uninstall opencv-python -y
!pip install opencv-python

In [ ]:
#Install DNN library 
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

## 4.3 Train Model

In [ ]:
# Train the model.
%cd {BASE_PATH}
#run the cell to start model training 

!python model_main_tf2.py --model_dir={MODEL_PATH +'/'+CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH +'/'+CUSTOM_MODEL_NAME}/pipeline.config
#!python model_main_tf2.py --model_dir=workspace/models/my_ssd_mobnet2 --pipeline_config_path=workspace/models/my_ssd_mobnet2/pipeline.config
# !python model_main_tf2.py --model_dir=models/[name_of_pre-trained-model_you_downloaded] --pipeline_config_path=models/[name_of_pre-trained-model_you_downloaded]/pipeline.config

## **4.4 compress model**

In [ ]:
# kompress model
%cd {MODEL_PATH}
!zip -r {OUTPUT_MODEL}"TRAIN_HARI.zip" {CUSTOM_MODEL_NAME}

In [ ]:
pwd

Congratulations! You have finished model training!

# **6.Export Trained Model.**

In [ ]:
OUTPUT_MODEL

In [ ]:
#Step 12- Export the Trained Model.
%cd {BASE_PATH}
#run the cell to start model training 
#!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path {BASE_PATH}/workspace/models/my_ssd_mobnet2/pipeline.config --trained_checkpoint_dir /content/gdrive/MyDrive/3.gesture_bahasa_isyarat/Tensorflow/workspace/models/my_ssd_mobnet2 --output_directory workspace/exported-models/MYMODELSSD4
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path {MODEL_PATH +'/'+CUSTOM_MODEL_NAME}/pipeline.config --trained_checkpoint_dir {MODEL_PATH +'/'+CUSTOM_MODEL_NAME} --output_directory {EXPORTED_MODELS}/{OUTPUT_MODEL}
# !python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/[name_of_pre-trained-model you downloaded]/pipeline.config --trained_checkpoint_dir ./models/[name_of_pre-trained-model_you_downloaded]/ --output_directory ./exported-models/my_model

#  **7.Testing  Model.**

We have finished training and exporting our model. It's time to test our model!

In [ ]:
# 5.1 Loading the saved_model

#Loading the saved_model
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL= EXPORTED_MODELS+'/'+OUTPUT_MODEL+'/saved_model'

print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')

In [ ]:
#CEK MODEL
PATH_TO_SAVED_MODEL

In [ ]:
# 5.2 Testing the Model.

#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+"/label_map.pbtxt",use_display_name=True)

#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)

In [ ]:
#Loading the image
img=[
      IMAGE_PATH+'/image_testing_rondi/SENIN/image_RONDI_215.jpg',
      IMAGE_PATH+'/image_testing_rondi/SELASA/image_RONDI_269.jpg',
      IMAGE_PATH+'/image_testing_rondi/RABU/image_RONDI_233.jpg',
      IMAGE_PATH+'/image_testing_rondi/KAMIS/image_RONDI_2192.jpg',
      IMAGE_PATH+'/image_testing_rondi/JUMAT/image_RONDI_2260.jpg',
      IMAGE_PATH+'/image_testing_rondi/SABTU/image_RONDI_2321.jpg',
      IMAGE_PATH+'/image_testing_rondi/MINGGU/image_RONDI_2356.jpg',
     ]
print(img)

#list containing paths of all the images

## 5.1.Show image testing

In [ ]:
#Step 16- show image testing

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

for image_path in img:

    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)

    

    
    %matplotlib inline
    plt.figure()
    plt.imshow(image_np)
    print('Done')
    plt.show()

## 5.2 Show Deteksi on image 

In [ ]:
%cd {IMAGE_PATH}/output_deteksi_image
dir_img_detection =str(NUM_STEPS)
!mkdir {dir_img_detection}

In [ ]:
#Step 17- Running the Inference.
import cv2
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

for image_path in img:

    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()
    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor=tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections=detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}
    detections['num_detections']=num_detections

    # detection_classes should be ints.
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    image_np_with_detections=image_np.copy()
       

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=1,     #max number of bounding boxes in the image
          min_score_thresh=.3,      #min prediction threshold
          agnostic_mode=False,
          line_thickness=3,
          keypoint_edges=4,
          )
    
    %matplotlib inline
    name= os.path.basename(image_path)
    plt.figure(figsize=[10,10],dpi=100)
    plt.imshow(image_np_with_detections)
    cv2.imwrite(IMAGE_PATH+'/output_deteksi_image/'+dir_img_detection+'/'+name, image_np_with_detections)
    print('Done')
    plt.show()

In [ ]:
pwd

Acknowledgements:

Huge Thanks to [Lyudmil Vladimirov](http://pcwww.liv.ac.uk/~sglvladi/) for allowing me to use some of the content from their amazing [TensorFlow 2 Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/index.html) for Local Machines!

Link to their [GitHub Repository](https://github.com/sglvladi/TensorFlowObjectDetectionTutorial).

# **8.TESTING WITH RECORD VIDEO**

In [ ]:
# from IPython.display import display, Javascript,HTML
# from google.colab.output import eval_js
# from base64 import b64decode

# def record_video(filename):
#   js=Javascript("""
#     async function recordVideo() {
#       const options = { mimeType: "video/webm; codecs=vp9" };
#       const div = document.createElement('div');
#       const capture = document.createElement('button');
#       const stopCapture = document.createElement("button");
      
#       capture.textContent = "Start Recording";
#       capture.style.background = "yellow";
#       capture.style.color = "black";

#       stopCapture.textContent = "Stop Recording";
#       stopCapture.style.background = "red";
#       stopCapture.style.color = "white";
#       div.appendChild(capture);

#       const video = document.createElement('video');
#       const recordingVid = document.createElement("video");
#       video.style.display = 'block';

#       const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
    
#       let recorder = new MediaRecorder(stream, options);
#       document.body.appendChild(div);
#       div.appendChild(video);

#       video.srcObject = stream;
#       video.muted = true;

#       await video.play();

#       google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

#       await new Promise((resolve) => {
#         capture.onclick = resolve;
#       });
#       recorder.start();
#       capture.replaceWith(stopCapture);

#       await new Promise((resolve) => stopCapture.onclick = resolve);
#       recorder.stop();
#       let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
#       let arrBuff = await recData.data.arrayBuffer();
      
#       // stop the stream and remove the video element
#       stream.getVideoTracks()[0].stop();
#       div.remove();

#       let binaryString = "";
#       let bytes = new Uint8Array(arrBuff);
#       bytes.forEach((byte) => {
#         binaryString += String.fromCharCode(byte);
#       })
#     return btoa(binaryString);
#     }
#   """)
#   try:
#     display(js)
#     data=eval_js('recordVideo({})')
#     binary=b64decode(data)
#     with open(filename,"wb") as video_file:
#       video_file.write(binary)
#     print(f"Finished recording video at:{filename}")
#   except Exception as err:
#     print(str(err))

# # Set path to save video here
# video_path = IMAGE_PATH+"/video/input/rondi_record.mp4"
# record_video(video_path)


## 8.1 DETEKSI HASIL VIDEO

In [ ]:
# import os
# import time
# import tensorflow as tf
# import cv2
# import numpy as np

# # from utils import label_map_util
# from PIL import Image
# from google.colab.patches import cv2_imshow
# from object_detection.utils import label_map_util
# from object_detection.utils import visualization_utils as viz_utils

# from IPython.display import HTML
# from base64 import b64encode

# #Path to saved model  

# PATH_TO_SAVED_MODEL = EXPORTED_MODELS+'/'+OUTPUT_MODEL+'/saved_model'

# # Load label map and obtain class names and ids
# #label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
# category_index=label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+"/label_map.pbtxt",use_display_name=True)


# def visualise_on_image(image, bboxes, labels, scores, thresh):
#     (h, w, d) = image.shape
#     for bbox, label, score in zip(bboxes, labels, scores):
#         if score > thresh:
#             xmin, ymin = int(bbox[1]*w), int(bbox[0]*h)
#             xmax, ymax = int(bbox[3]*w), int(bbox[2]*h)

#             cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0,255,0), 2)
#             cv2.putText(image, f"{label}: {int(score*100)} %", (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
#     return image

# if __name__ == '__main__':
    
#     # Load the model
#     print("Loading saved model ...")
#     detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
#     print("Model Loaded!")
    
#     # Video Capture (video_file)
#     video_capture = cv2.VideoCapture(IMAGE_PATH+"/video/input/rondi_record_7HARI.mp4")
#     start_time = time.time()
    
#     frame_width = int(video_capture.get(3))
#     frame_height = int(video_capture.get(4))
#     #fps = int(video_capture.get(5))
#     size = (frame_width, frame_height)
    
#     #Initialize video writer
#     result = cv2.VideoWriter(IMAGE_PATH+"/video/output/rondi_record_detection.avi", cv2.VideoWriter_fourcc(*'MJPG'),15, size)

#     while True:
#       ret, frame = video_capture.read()
#       if not ret:
#           print('Unable to read video / Video ended')
#           break
    
#       frame = cv2.flip(frame, 1)
#       image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#       # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
#       # The model expects a batch of images, so also add an axis with `tf.newaxis`.
#       input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

#       # Pass frame through detector
#       detections = detect_fn(input_tensor)

#       # Set detection parameters

#       score_thresh = 0.4   # Minimum threshold for object detection
#       max_detections = 1

#       # All outputs are batches tensors.
#       # Convert to numpy arrays, and take index [0] to remove the batch dimension.
#       # We're only interested in the first num_detections.
#       scores = detections['detection_scores'][0, :max_detections].numpy()
#       bboxes = detections['detection_boxes'][0, :max_detections].numpy()
#       labels = detections['detection_classes'][0, :max_detections].numpy().astype(np.int64)
#       labels = [category_index[n]['name'] for n in labels]

#       # Display detections
#       visualise_on_image(frame, bboxes, labels, scores, score_thresh)

#       end_time = time.time()
#       fps = int(1/(end_time - start_time))
#       start_time = end_time
#       cv2.putText(frame, f"FPS: {fps}", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
#       #cv2_imshow(frame)
      
#       #Write output video
#       result.write(frame)

#     video_capture.release()


## 8.2 CONVERSI TO WEBM

In [ ]:
# %cd {IMAGE_PATH}/video/output
# !ffmpeg -i ./rondi_record_detection.avi -vcodec vp9 ./rondi_record_detection.webm -y


In [ ]:
# # Displaying the Drawing Result
# %cd {IMAGE_PATH}/video/output

# import io
# from base64 import b64encode
# from IPython.display import HTML

# with  io.open('rondi_record_detection.webm','r+b') as f:
#     mp4 = f.read()
# data_url = "data:video/webm;base64," + b64encode(mp4).decode()
# HTML("""
# <video width=200 controls>
#       <source src="%s" type="video/webm">
# </video>
# """ % data_url)

# **9.COMPRESS PROJECT**

In [ ]:
# %cd /content/gdrive/MyDrive
# !zip -r {NEW_folder_work}_{NUM_STEPS}"TRAIN_HARI.zip" {NEW_folder_work}
# # !unzip "/content/gdrive/MyDrive/"{NEW_folder_work}"_40K_TRAIN.zip" -d "/content/gdrive/MyDrive/"{NEW_folder_work}
# # !mv /content/gdrive/MyDrive/{NEW_folder_work}/{NEW_folder_work} /content/gdrive/MyDrive/{NEW_folder_work}_COPY